In [ ]:
# Consolidate Sheets in Streamlit.
# Have Read Me and Clear DOcumented process to remind self.

# Consolidate Notes such that it's also exportable and ordered.

## Need a process for Looking at a .py File and then ensuring I have a reference usage here so I can remember.
## Need to also Classify the models based on usage and Dates so I can see changes.
## Need to determine a plan to migrate to a single usage.

# Add Filter on Definition Summary

In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

#!bash /Users/derekdewald/scripts/daily_shell_job.sh

In [ ]:
from daily_processes import daily_test

#review_test_results()
daily_test()
review_test_results()

#df = pd.read_csv('/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv')
#df

In [ ]:
words = []

from data_d_strings import gpt_question
gpt_question(words)

In [ ]:
from daily_processes import generate_dictionary



In [10]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from shared_folder import read_directory,text_file_import,parse_dot_py_file
from input_functions_ignore import input1,input2,input3
import data_d_dicts,data_d_lists,data_d_strings
from data_d_dicts import function_table_dictionary,links
from dict_processing import dict_to_dataframe
from list_processing import list_to_dataframe
from data_validation import column_segmenter

def generate_dictionary(notes_df=None,
                        definition_df=None,
                        export_location=''):

    '''
    Function Used to Generate D Learning Notes, which is a consolidated view of D Notes and D Definitions.
    Used as a crtical input to D Streamlit Dashboard.

    CSV Generated from this process saved to Github and Used for Streamlit Dashboard.

    Approach takes D Notes as the Primary Source and then merges in Definitions, based on a 4 step integration process.

    Approach uses some Filtering Logic
    
    Step1: 
        When Word in Both Sheets is Identical, then utilize word as definition.
        If Word Does not have a Definition in the D Notes, then replace blank with the definition. 
        If word has a definition, then add a new record.
        
    Step2:
        Utilzing Function notes_word_equals_word_definition_categorization, integration where notes.word=definitions.categorization
        It also begins to replace Word.
        
    Step3:
        Utilizing Function notes_word_equals_definition_process, integration where notes.word=definitions.process
        
    Step4: 
        Merge all other Definitions into Notes Sheet. 

    Parameters:
        notes_df (df): If Blank, then Function will generate call
        definitions_df(df): If Blank Function will call.
        export_Location (str): If populated, location where .csv file will be saved.

    Returns:
        Object Type

    date_created:12-Jan-26
    date_last_modified: 12-Jan-26
    classification:TBD
    sub_classification:TBD
    usage:
        notes_df,examine_further= generate_dictionary()
        notes_df,insert_records,insert_records2,insert_records3,examine_further= generate_dictionary()


    
    '''
    
    def notes_word_equals_word_definition_categorization(df,notes_df):
        df = df.copy()
        # Create Definition where Definition Categorization Equals Word.
        df['WORD_IS_ZATION'] = np.where(df['Categorization_DEF'].isin(notes_df['Word'].unique().tolist()),1,0)
        
        merge_ = df[df['WORD_IS_ZATION']==1].copy()
        residual_values= df[df['WORD_IS_ZATION']!=1].copy()
        
        merge_ = merge_[['Categorization_DEF','Word','Definition_DEF']].copy()
        merge_['Definition'] = merge_['Word'] + ": " + merge_['Definition_DEF']
        merge_ = merge_.drop(['Definition_DEF','Word'],axis=1).rename(columns={'Categorization_DEF':"Word"})
        merge_['Categorization'] = 'Definition'
    
        merge_ = merge_.merge(notes_df.drop_duplicates('Word')[['Word','Process']],on='Word',how='left')

        residual_values = residual_values[['Process_DEF','Categorization_DEF','Word','Definition_DEF']].rename(columns={'Process_DEF':'Process',
                                                                                             'Categorization_DEF':'Categorization',
                                                                                             'Definition_DEF':'Definition'})
        return merge_,residual_values

    def notes_word_equals_definition_process(examine_further,notes_df):
        '''
            
        '''
    
        df = examine_further.copy()
        notes_df = notes_df.copy()
    
        # Make List
        notes_word_list = notes_df['Word'].unique().tolist()
    
        examine_further = df[~df['Process'].isin(notes_word_list)].copy()
        insert_df = df[df['Process'].isin(notes_word_list)].copy()
    
        # Clean Up Insert DF so it meets Notes DF Structure, Not Definition DF Structure.
        # Definition Updated to Include Word
        # Word is Updated to Include Process.
        # Categorization Does not Change
        # Process Takes on Whatever Value in Notes is, as this is now a input into that Process.
        
        # Word is Updated to include Categorization (needs to happen After Definition is Change)
        
        insert_df['Definition'] = insert_df['Word'] + ": " +  insert_df['Definition'] 
        insert_df['Word'] = insert_df['Process'].copy()
        insert_df.drop(['Process'],axis=1,inplace=True)
        insert_df = insert_df.merge(notes_df.drop_duplicates('Word')[['Word','Process']],on='Word',how='left')
    
        return examine_further,insert_df

        
    from data_d_dicts import links

    try:
        notes_df = notes.copy()
    except:
        notes_df = pd.read_csv(links['google_notes_csv'])
        
    try:
        definition_df = definition_df.copy()
        definition_df.rename(columns={'Process':'Process_DEF','Categorization':"Categorization_DEF",'Definition':"Definition_DEF"},inplace=True)
    except:
        definition_df = pd.read_csv(links['google_definition_csv']).rename(columns={'Process':'Process_DEF','Categorization':"Categorization_DEF",'Definition':"Definition_DEF"})
        definition_df.sort_values(['Process_DEF','Categorization_DEF','Word'],inplace=True)
        
    # Step 1: Merge Definitions into Words where they explicitly Match. No logic required.
    # Identify Where there is a Record. 
        # Example 1: ML Project >> Process Step >> Problem Definition
            # This is a Definition to a process which has Steps. Need to Merge a NEW RECORD.
        # Example 2: Best Linear Unbiased Estimator
            # This is a Example which needs Definitions Merged in, some of which arent direct Definitions. 
    
    # We will create a modified Definition DF. To identify where direct matches exist.
    # Need to distribute information from temp_def until Empty. Need Data Quality View Steps.

    temp_def = definition_df[['Process_DEF','Categorization_DEF','Word','Definition_DEF']].merge(notes_df[['Process','Categorization','Word','Definition']].drop_duplicates('Word'),on='Word',how='left',indicator=True)
    merge = temp_def[temp_def['_merge']=='both'].drop('_merge',axis=1)
    examine_further = temp_def[temp_def['_merge']!='both'].drop('_merge',axis=1)
    # Naming for trouble shooting

    # Two Types of Merge
    # Inserting New Records, specifically Where the existing Note has a Definition, this means there is an existing Process. So this
    # Value Represents a Definition, and there is a New Record Insert

    # Merging Definition. WHere the existing note has No Definition, then we will incorporate a definition. This technically could be 
    # Ignored, but by doing this, I can indirectly influence Order without direct assignment by assuming the Notes Order is Reference Point.

    # Insert Record
    insert_records = merge[merge['Definition'].notnull()].drop(['Process_DEF','Categorization_DEF','Definition','Categorization'],axis=1).rename(columns={'Definition_DEF':"Definition"})
    insert_records['Categorization'] = 'Definition'
    insert_records['Source'] = 'Insert Records'
    
    merge_records  = merge[merge['Definition'].isnull()]
    
    # WAIT TO MERGE UNTIL END IF POSSIBLE. THAT WAY I ONLY NEED TO RANK ONCE.
    
    notes_df =  notes_df.merge(merge_records[['Word','Definition_DEF']],on='Word',how='left')
    notes_df['Definition'] = notes_df['Definition'].fillna(notes_df['Definition_DEF'])
    notes_df.drop('Definition_DEF',axis=1,inplace=True)
    notes_df['Source'] = 'Notes DF'
    #### Step 2: 

    # Insert Instances where Word In Notes is Equal to Categorization in Definition. 
    # When a Process as Steps which are defined and need to be included, but Also Represent a Definition unto themselves.
    # Example Bias, Bias Variance Trade Off, they are STEPS to in the Feature Selection Process, but also key terms which 
    # Deserve their own definition and explanation.

    insert_records2, examine_further = notes_word_equals_word_definition_categorization(examine_further,notes_df)
    insert_records2['Source'] = 'Insert Records2'
    # Start To Bring Ordering to the Data Set.

    c1_df = notes_df[['Process']].drop_duplicates().reset_index(drop=True).assign(C1_ORDER=lambda df: df.index+1)
    c12_df = notes_df[['Process','Categorization']].drop_duplicates().reset_index(drop=True).assign(C12_ORDER=lambda df: df.index+1)
    c3_df = notes_df[['Word']].drop_duplicates().reset_index(drop=True).assign(C3_ORDER=lambda df: df.index+1)

    examine_further,insert_records3 = notes_word_equals_definition_process(examine_further,notes_df)
    insert_records3['Source'] = 'Insert Records3'
    examine_further['Source'] = 'Examine Further'
    
    # Determine Order for Sorting.
    notes_df = notes_df.merge(c1_df,on='Process',how='left')
    notes_df = notes_df.merge(c3_df,on=['Word'],how='left')

    insert_records = pd.concat([
        insert_records,
        insert_records2,
        insert_records3,
        examine_further
    ])

    # Determine COL1 and COL3 Sort Order
    insert_records = insert_records.merge(c1_df,on='Process',how='left')
    insert_records = insert_records.merge(c3_df,on='Word',how='left')

    # Combine ALl DF
    notes_df = pd.concat([
        notes_df,
        insert_records
    ])

    d_notes_categorization_order = ['Definition',
                                'Guiding Principle',
                                    
                                'Algorithm',
                                'Consideration',
                                'Process Step']
    
    residual_list = [x for x in notes_df['Categorization'].unique() if (x not in d_notes_categorization_order) & (pd.notna(x))]
    d_notes_categorization_order.extend(residual_list)

    print(d_notes_categorization_order)
    
    # Determine COL2 Sort Order, which is based on Mapping and Can be done once when consolidated.
    col2_order_dict = {d_notes_categorization_order[x]:x+1 for x in range(len(d_notes_categorization_order))}
    notes_df['C2_ORDER'] = notes_df['Categorization'].map(col2_order_dict)
    notes_df['C2_ORDER'] = notes_df['C2_ORDER'].fillna(1000)   

    notes_df =  notes_df.sort_values(['C1_ORDER','C2_ORDER','C3_ORDER'])

    # Need to insure that every record 
    
    #return notes_df,insert_records,insert_records2,insert_records3,examine_further
  
    notes_df = notes_df.drop(['C1_ORDER','C3_ORDER','C2_ORDER'],axis=1)

    if export_location:
        notes_df.to_csv(export_location,index=False)

    return notes_df,examine_further


notes_df, examine_further = generate_dictionary()

['Definition', 'Guiding Principle', 'Algorithm', 'Consideration', 'Process Step', 'Model Type', 'Process', 'Requirement', 'Project Implementation', 'Project Conceputalization', 'Expected Outcomes', 'Functional Role', 'Regularization', 'Semantic Type', 'Function', 'Model Architecture', 'Optimizer', 'Procedure', 'Parameter', 'Automating Python', 'Tooling', 'Evaluation', 'Pipeline Stage', 'Theorem', 'Concept', 'Hadoop', 'Paradigm', 'Spark']


In [12]:
notes_df['Categorization'].value_counts()

Categorization
Algorithm                    211
Definition                   125
Process Step                  48
Consideration                 40
Guiding Principle             33
Procedure                     22
Parameter                     19
Semantic Type                 16
Spark                          8
Project Conceputalization      7
Project Implementation         7
Optimizer                      7
Model Type                     6
Requirement                    5
Theorem                        4
Function                       4
Automating Python              3
Regularization                 3
Functional Role                3
Process                        3
Pipeline Stage                 2
Model Architecture             2
Expected Outcomes              1
Concept                        1
Hadoop                         1
Paradigm                       1
Evaluation                     1
Tooling                        1
Name: count, dtype: int64

In [11]:
notes_df[notes_df['Process'].fillna("").str.contains('Clu')]

,Process,Categorization,Word,Definition,Source
70,Clustering,Guiding Principle,Behavioural Clustering Goal,"Must Clarify this clearly, specifically the di...",Notes DF
71,Clustering,Guiding Principle,Behavioural Clustering Goal,"Small Number of Stable, Interpretable, Member ...",Notes DF
72,Clustering,Guiding Principle,Expected Outcomes,Stability over Uniqueness,Notes DF
73,Clustering,Guiding Principle,Expected Outcomes,Interpretability over novelty,Notes DF
74,Clustering,Guiding Principle,Expected Outcomes,Repeatability over perfection.,Notes DF
117,Clustering,Guiding Principle,Expected Outcomes,Consistent,Notes DF
118,Clustering,Guiding Principle,Expected Outcomes,Repeatable,Notes DF
115,Clustering,Guiding Principle,What is it?,"Calculation, utilizing mathematical precision ...",Notes DF
116,Clustering,Guiding Principle,How to Decide?,"Understand the impact of components, can preve...",Notes DF
119,Clustering,Guiding Principle,Benefit,Strategic Conversations become diagnostic not ...,Notes DF
